In [15]:
import xml.etree.ElementTree as ET
import csv
import os
import datetime
import shutil
import paramiko

In [16]:

# SFTP connection parameters
host = "sftp-partners.glovoapp.com"
port = 22
username = "realco-it"
password = "FI5MC3Yt2RLeyPo7HNY6jswD"
folder = "/Users/michelemaestrale/Desktop/GLOVO/REALCO/flow"

server = paramiko.SSHClient()
server.set_missing_host_key_policy(paramiko.AutoAddPolicy())
server.connect(hostname = host, username = username, password = password, port = port)
ftp = server.open_sftp()


localpath = '/Users/michelemaestrale/Desktop/GLOVO/REALCO/flow'
remotepath = '/glovoapp-partners-sftp-bucket-121e9009/home/realco-it/Input_realco-it'

for file in ftp.listdir(remotepath):
    ftp.get(remotepath+'/'+file, localpath+'/'+file)

ftp.close()
server.close()

In [17]:
#lettura del file di menu:
product_list = []

for file in os.listdir(folder):
    # Check if file is a .xml file with "cedi" in its name
    if file.endswith('.xml') and 'cedi' in file:
        # Parse the XML file
        tree = ET.parse(os.path.join(folder, file))
        root = tree.getroot()
        for child in root:
            for subchild in child:
                if subchild.tag == 'ProductId':
                    product_list.append(subchild.text)

In [14]:
# print(product_list)

In [21]:
#nuova to test

# list to store the result of each file transformation
results = []

# iterate through all files in the folder
for filename in os.listdir(folder):
    # check if the file is an xml file and if the file name contains 'PDV'
    if filename.endswith(".xml") and 'PDV' in filename:
        # parse xml file
        try:
            tree = ET.parse(os.path.join(folder, filename))
            root = tree.getroot()

            # get store_name, store and timestamp
            store_name = 'Realco'
            store = root.attrib["PDV"]
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

            # create the filename
            filename = f"{store_name}_{store}_{timestamp}.csv"

            # open a csv file for writing
            with open(os.path.join(folder, filename), 'w', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow(["store_id","product_id","price","stock"])

                # get store_id from xml
                store_id = root.attrib["PDV"]
                filtered_product = product_list
                

                # iterate over all articles
                for article in root:
                    
                    product_id = article.find("ProductId").text
                    filtered_product = [x for x in filtered_product if x != product_id]
                    price_promo = article.find("PricePromo").text
                    price = article.find("Price").text

                    # set price to PricePromo if it's not equal to 0, otherwise set it to Price
                    if price_promo != "0.00":
                        price = price_promo
                    else:
                        price = price
                    # set stock to 10 for all records
                    stock = 10
                    if float(price)>0:
                        # write to csv
                        csvwriter.writerow([store_id, product_id, price, stock])
                for id in filtered_product:
                    csvwriter.writerow([store_id, id, 9999, 0])
                    
                
            results.append((filename, True))
        except:
            results.append((filename, False))

print(results)



[('Realco_3960_2023-01-20_14-45-52.csv', True), ('Realco_1111_2023-01-20_14-45-53.csv', True), ('Realco_3960_2023-01-20_14-45-55.csv', True)]


In [22]:
# SFTP connection parameters
host = "sftp-partners.glovoapp.com"
port = 22
username = "realco-it"
password = "FI5MC3Yt2RLeyPo7HNY6jswD"

server = paramiko.SSHClient()
server.set_missing_host_key_policy(paramiko.AutoAddPolicy())
server.connect(hostname = host, username = username, password = password, port = port)
ftp = server.open_sftp()


localpath = '/Users/michelemaestrale/Desktop/GLOVO/REALCO/flow'
remotepath = '/glovoapp-partners-sftp-bucket-121e9009/home/realco-it/Input_realco-it'

for file in os.listdir(localpath):
    if file.endswith('.csv'):
        ftp.put(localpath+'/'+file, remotepath+'/'+file)

ftp.close()
server.close()

In [5]:

source = '/Users/michelemaestrale/Desktop/GLOVO/REALCO/flow'
destination = '/Users/michelemaestrale/Desktop/GLOVO/REALCO/flow/old'

for f in os.listdir(source):
    if os.path.exists(destination+'/'+f):
        os.remove(destination+'/'+f)
    shutil.move(source+'/'+f, destination)

In [24]:
import shutil
import os

#csvpath = f"""/Users/michelemaestrale/Desktop/GLOVO/Integration_dive_deep/{store_name}/csvtozero"""
## UP TO DATE
#csvpath = f"""/Users/michelemaestrale/Desktop/GLOVO/Integration_dive_deep/{store_name}/uptodate"""

# Set the source and destination folder paths
src_folder = '/Users/michelemaestrale/Desktop/GLOVO/REALCO/flow'
dst_folder = '/Users/michelemaestrale/Desktop/GLOVO/REALCO/flow/old'

# Iterate through all files in the source folder
for file_name in os.listdir(src_folder):
    src_file = os.path.join(src_folder, file_name)
    dst_file = os.path.join(dst_folder, file_name)

    # Move the file to the destination folder (replace if already present)
    if os.path.isfile(src_file):
        # Move the file to the destination folder (replace if already present)
        shutil.move(src_file, dst_file)

    # Set the source and destination folder paths
